# Tests to be implemented in github actions. 

In [3]:
import sys
sys.path.append("../../../xapres_package/")
import xapres_package as xa
import numpy


## Test of file name/number selection in xapres.load_all

In [7]:
# test that we can select and load the same file using two different methods: 1) selecting the file number, 
# and 2) using the filename that we get from 1) to load the same file again.
def test_file_selection():
    directory='gs://ldeo-glaciology/GL_apres_2022/A101'
    xa1 = ApRESDefs.xapres(max_range=1400)
    xa1.load_all(directory, 
                remote_load = True,
                file_numbers_to_process=[0,1])

    xa2 = ApRESDefs.xapres(max_range=1400)
    xa2.load_all(directory, 
                remote_load = True,
                file_names_to_process = xa1.dat_filenames_to_process)

    assert xa1.data.identical(xa2.data)

In [8]:
test_file_selection()

## Test the loading of a single dat file from the google bucket.


In [18]:
#import pytest
#from xapres_package import ApRESDefs

def test_dat_file_loading():
    directory='gs://ldeo-glaciology/GL_apres_2022/A101'
    fs = xa.load.from_dats(max_range=1400)
    fs.load_all(directory, 
                remote_load = True,
                file_numbers_to_process=[0],
                bursts_to_process=[0])
    #return fs
    assert numpy.isclose(fs.data.chirp.mean().values, 0.02611298) 

In [20]:
test_dat_file_loading()

## Test the file search code
Make sure that the 'dat' is not case sensitive and that the search suffix option works as intended

In [23]:
def test_file_search():
    fs = xa.load.from_dats()
    data_directory = '../../data'
    
    higher_level_list_of_dats = fs.list_files(data_directory + "/sample")
    # this checks that the list of files is not empty
    assert higher_level_list_of_dats  

    lower_level_list_of_dats = fs.list_files(data_directory + "/sample/polarmetric")
    # test that all the files found in a lower level directory were also found when searching in a higher level directory
    assert all(item in higher_level_list_of_dats for item in lower_level_list_of_dats)

    # test that the case of the extension (DAT vs dat) doesnt matter
    assert len(fs.list_files(data_directory + "/sample/different_case_examples")) == 2

    # test the search_suffix option is working
    assert len(fs.list_files(data_directory + "/sample/polarmetric", search_suffix='HH')) == 1

In [24]:
test_file_search()

## Test file selection methods

In [32]:
## Use two different methods for selecting the same ApRES DAT file from a google bucket. 
#  In each case load it and then check that we have loaded the correct file. 

def test_file_selection_methods():
    directory='gs://ldeo-glaciology/GL_apres_2022/A101'
    fs1 = xa.load.from_dats(max_range=1400)
    fs1.load_all(directory, 
                remote_load = True,
                file_numbers_to_process=[0,1])

    fs2 = xa.load.from_dats(max_range=1400)
    fs2.load_all(directory, 
                remote_load = True,
                file_names_to_process = fs1.dat_filenames_to_process)

    assert fs1.data.identical(fs2.data)

In [33]:
test_file_selection_methods()

## Test polarmetric loading

In [40]:
## test polarmetric local loading by loading the same waypoint twice as if it is two different ones and chaeking 
# that you get the same thing twice.

# tests the attended option and the polarmetric option from locally stored dat files (as opposed to cloud stored dat files)

def test_polarmetric_load():
    
    fs = xa.load.from_dats()
    fs.load_all(attended=True, 
                directory=["../../data/sample/polarmetric", "../../data/sample/polarmetric"], 
                polarmetric=True)
    
    assert len(fs.data.waypoint) == 2
    assert all(fs.data.isel(waypoint=0).filename.values == fs.data.isel(waypoint=1).filename.values)


In [41]:
test_polarmetric_load()

## Test `generate_xarray` and `load_zarr` wrappers

In [45]:
#  Test `generate_xarray` and `load_zarr` wrappers
def test_wrappers():
    
    from_DAT_unattended = xa.load.generate_xarray(directory='../../data/sample/single_dat_file/', 
                file_numbers_to_process = [0], 
                bursts_to_process=[0],
                )

    from_zarr = xa.load.load_zarr()


In [46]:
test_wrappers()